Questa sezione tratta la correlazione delle statistiche. 
Lo scopo è quello di capire quali dati i sono meglio e peggio correlati fra loro. Vogliamo quindi trovare quali sono le statisitche che si relazionano meglio con quei dati che sono i più importanti per un attaccante, come ad esempio il numero di Goal, Assist oppure il numero di Chance da Goal create. L'idea dietro a questa analisi è quella di capire quali possono essere le statistiche (diverse da quelle chiave sopracitate) che una squadra può andare a guardare per trovare un attaccante d'impatto per la propria squadra.

In [ ]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,'attaccanti_19.xlsx': 2019}


In [ ]:
# Questa sezione è divisa in tre parti, in tre celle diverse. 

# La prima cella si occupa esclusivamente 
# delle tabelle di correlazione infatti, una volta scaricati i file si potranno visualizzare cinque quadrati interattivi
# che riportano come si correlano tra di loro le statistiche selezionate nel dataframe. Alla fine per comodità verranno
# riportate le colonne usate.

# Nella seconda cella abbiamo inserito una funzione che ha come scopo quello di scoprire quali sono le 5 statistiche meglio e peggio
# correlate con quella desiderata data in input.

# Nella terza cella abbiamo inserito un'altra funzione che però ha come scopo quello di vedere come sono correlate
# due statistiche specifiche, date in input alla funzione, e vedere come la loro correlazione è cambiata negli ultimi 5 anni,
# sicuramente utile per scoprire se la correlazione o scorrelazione di due dati sia casuale o una cosa costante nel tempo.

Prima di tutto bisogna fare un processo di 'data cleaning' ovvero una pulizia del dataframe, aggiungendo colonne che possono essere utili oppure togliendone altre che non voglio prendere in considerazione.
 
Ovviamente scartiamo le celle relative al Nome del giocatore, la Squadra per cui gioca e la sua Nazionalità.
Subito dopo tolgiamo quelle colonne che rappresentano "operazioni" fra altre statistiche, come ad esempio somme fra due dati, percentuali o rapporti, in quanto non sono statistiche di gioco (realizzate sul campo dal giocatore) ma create ad hoc per altri tipi di analisi non inerenti al nostro.

In [ ]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche per annata

correlations = []
years = []

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] # Creo la colonna dei duelli aerei, può essere utile per l'analisi

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")  # Ottengo in output il grafico di correlazione
    fig.show()

# Inserisco questa riga di codice in modo da poter visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')


Otteniamo in output i 5 grafici richiesti relativi alle 5 stagioni prese in considerazione. La barra sulla destra indica l'indice di correlazione in base al colore del quadrato relativo a due colonne. Muovendo il cursore sulle colonne si può vedere le due statistiche prese in considerazione ed il loro indice di correlazione. 

La presenza dei colori ci aiuta subito a notare che i grafici si assomigliano molto, nonostante vengano presi in considerazione 5 anni diversi nei quali hanno giocato molti giocatori diversi stagione per stagione. 
Immaginiamo quindi che gli indici di correlazione rimangano abbastanza costanti durante gli anni. Questo ci fa molto comodo, in quanto ci rassicura sul fatto che i risultati che troveremo non saranno casuali ma validi nel corso degli anni.

Oltre a questa osservazione possiamo anche notare, riconoscendo i colori, che ci sono statistiche mal correlate con tutte le altre. Salta subito all'occhio la colonna relativa ai Minuti giocati, indicata con 'Min'.

In [ ]:
# Creo una funzione che mi permette di scoprire quali sono le statistiche meglio e peggio correlate con una desiderata

def best_worst(stat):
 for file in files:
    year = year_mapping[file]
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True) 

    corr_matrix = data_frame.corr()

    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni (cambiare i range per saperne di più o di meno)
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset. Controlla l'input")


In [ ]:
stat = 'Min'
best_worst(stat)

Non c'è bisogno di richiamare tutti i valori, ci basta usare la funzione per confermare ciò che pensavamo. 
Nel corso degli anni la statistica meglio correlata con quella dei Minuti giocati risulta essere 'Age' con un indice di 0.144865 (abbastanza basso) registrato nel 2023. Inoltre notiamo come nelle 5 statistiche meglio correlate durante gli anni, la maggior parte abbiano addirittura un indice negativo. Questo è sicuramente il primo risultato grosso ed inaspettato di questa analisi in quanto ci aspettavamo che un minutaggio elevato avesse un grosso impatto sulle statistiche del singolo. 

Da un certo punto di vista però è un risultato che ci piace in quanto suggerisce che il rendimento di un attaccante non dipende banalmente da quanto gioca ma da come gioca. Tutto ciò da ancora più valore alla nostra ricerca.

In [ ]:
# Creo una seconda funzione che mi dice l'indice di correlazione di due statistiche specifiche
# ed il cambiamento di questo nel corso delle stagioni
def correlazione(stat1,stat2):
 correlations = []

 for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%',
       'G/Sh', 'G/SoT', 'G-xG','Cmp%','Succ%','Won%'], inplace=True)

    corr_matrix = data_frame.corr()

    year = year_mapping[file]

    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")
    else:
        correlations.append((year, None))
        print('NaN')

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2}", markers=True)
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(range = [-1,1], dtick = 0.10)
    fig.show()
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide. Controlla l'input")